# Tea or Coffee?

### Import Packages

In [180]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import seaborn as sns

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

### Extract Data from Wikipedia

In [181]:
#Use the function get from the requests package to get the data
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page = requests.get(link)
# Read the data into a pandas dataframe using the function read_html
wiki_raw = pd.read_html(wiki_page.content, header=0)[0]

#Modify the dataframe to remove all rows with a borough that is Not assigned.
wiki_new = wiki_raw[wiki_raw.Borough != 'Not assigned']

#Modify the dataframe to replace all rows with a Neighbourhood that is Not assigned to the name of the Borough
wiki_new.Neighbourhood.replace('Not assigned',wiki_new.Borough,inplace=True)
#Group the rows by the Postal Code.
toronto = wiki_new.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x))
#Reset the index
toronto = toronto.reset_index()
toronto.head()

C:\Users\sabee\anaconda3\lib\site-packages\pandas\core\generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Load Geographical CSV file and merge with Wikipedia Data using Postal Code

In [182]:
#Used the csv file to create the dataframe
geodata = 'http://cocl.us/Geospatial_data'
geo=pd.read_csv(geodata)
#Merge the two tables on the Postal Code field
toronto_merged = pd.merge(geo, toronto, on='Postal Code')
toronto_mdata = toronto_merged[['Postal Code','Borough','Neighbourhood','Latitude','Longitude']]
toronto_mdata.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Calculate the number of neighbourhoods in each Borough

In [183]:
# Number of neighbourhoods in each borough
toronto_mdata.groupby('Borough').count()['Neighbourhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighbourhood, dtype: int64

### Connect to Foursquare API

In [184]:
CLIENT_ID = 'WVC0VWTQJ3IDSJRAI4DW3KF5WB5GRKOJIVNQK3VAXQ11OHTU' # your Foursquare ID
CLIENT_SECRET = 'TLMAQ55KBYBUBPTGNOSITODOILHLUOUOBRPBVFIN3UMVO3TC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Reduce Original Dataframe to inclue Toronto neighbourhoods only

In [185]:
#Only Toronto Neighbourhoods
toronto_data=toronto_mdata[toronto_mdata['Borough'].str.contains("Toronto")]
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Collect Venue Data within a 500km radius

In [186]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [187]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )
toronto_venues.head()

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant


### Group venues by Neighbourhood

In [188]:
# Group venues by Neighbourhood
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,65,65,65,65,65,65
Christie,17,17,17,17,17,17
Church and Wellesley,76,76,76,76,76,76
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,34,34,34,34,34,34


### Get unique values and their names

In [189]:
#Get unique values and their names
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
print("The Venue Categories are", toronto_venues['Venue Category'].unique())

There are 235 uniques categories.
The Venue Categories are ['Trail' 'Health Food Store' 'Pub' 'Neighborhood' 'Asian Restaurant'
 'Cosmetics Shop' 'Ice Cream Shop' 'Greek Restaurant' 'Italian Restaurant'
 'Brewery' 'Juice Bar' 'Yoga Studio' 'Fruit & Vegetable Store'
 'Dessert Shop' 'Pizza Place' 'Restaurant' 'Bookstore' 'Grocery Store'
 'Café' 'Furniture / Home Store' 'Spa' 'Bubble Tea Shop' 'Coffee Shop'
 'Bakery' 'Caribbean Restaurant' 'Indian Restaurant' 'Frozen Yogurt Shop'
 'Lounge' 'Liquor Store' 'American Restaurant' 'Sushi Restaurant' 'Gym'
 'Fish & Chips Shop' 'Fast Food Restaurant' 'Park' 'Burrito Place'
 'Pet Store' 'Steakhouse' 'Movie Theater' 'Sandwich Place' 'Board Shop'
 'Fish Market' 'Seafood Restaurant' 'Gay Bar' 'Cheese Shop'
 'Middle Eastern Restaurant' 'Comfort Food Restaurant' 'Stationery Store'
 'Thai Restaurant' 'Coworking Space' 'Wine Bar'
 'Latin American Restaurant' 'Bar' 'Gastropub' 'Gym / Fitness Center'
 'Clothing Store' 'Bank' 'Convenience Store' 'Diner' 'S

### Determining whether there are Coffee shops in Toronto and how many

In [190]:
#Determining whether there are Coffee shops in Toronto
"Coffee Shop" in toronto_venues['Venue Category'].unique()

True

In [191]:
#Determining the number of Coffee shops in Toronto
coffeetoronto_venues = toronto_venues[toronto_venues['Venue Category'].str.contains("Coffee Shop")]
print(coffeetoronto_venues.shape)
coffeetoronto_venues[['Neighbourhood','Venue Category']].groupby('Neighbourhood').count().sort_values('Venue Category',ascending = False).head(10)

(158, 7)


,Venue Category
Neighbourhood,
"Toronto Dominion Centre, Design Exchange",14
"Commerce Court, Victoria Hotel",14
"First Canadian Place, Underground city",13
"Harbourfront East, Union Station, Toronto Islands",12
Stn A PO Boxes,11
Central Bay Street,11
"Queen's Park, Ontario Provincial Government",9
Church and Wellesley,8
"Regent Park, Harbourfront",8


### Convert Venue Category variable to Binary variables using One Hot Ecoding

In [192]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

### Calculate the mean occurence of a Venue by Neighbourhood

In [193]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,0.0,0.017544,0.000000,0.0,0.0,0.0,0.000000,0.017544,0.0,0.017544,0.035088,0.0,0.000000,0.0,0.017544,0.017544,0.0,0.035088,0.0,0.0,0.017544,0.0,0.0000,0.0,0.0000,0.0,0.017544,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.035088,0.0,0.0,0.0,0.0,0.035088,0.0,0.0,0.0,0.000000,0.017544,0.035088,0.087719,0.0,0.0,0.0,0.0,0.0,0.0,0.017544,0.000000,0.017544,0.000000,0.0,0.0,0.017544,0.0,0.0,0.0,0.0,0.017544,0.0,0.017544,0.000000,0.0,0.0,0.0,0.000000,0.0,0.017544,0.0,0.0,0.0,0.000000,0.035088,0.000000,0.0,0.0,0.017544,0.0,0.0,0.0,0.0,0.017544,0.017544,0.0,0.0,

### Create new dataframe with information on Coffee Shops only

In [194]:
coffee = toronto_grouped[["Neighbourhood","Coffee Shop"]]
coffee.head() #.head() shows the first twenty rows only

,Neighbourhood,Coffee Shop
0,Berczy Park,0.087719
1,"Brockton, Parkdale Village, Exhibition Place",0.083333
2,"Business reply mail Processing Centre, South C...",0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.062500
4,Central Bay Street,0.169231


### Cluster Neighbourhoods based on the mean occurences of Coffee Shops

#### Generate Clusters

In [195]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = coffee.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 0, 2, 1, 2, 1, 1, 2, 0])

#### Add Cluster labels to dataframe

In [196]:
# add clustering labels
toronto_coffee = coffee
toronto_coffee.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_coffee.head()

,Cluster Labels,Neighbourhood,Coffee Shop
0,2,Berczy Park,0.087719
1,2,"Brockton, Parkdale Village, Exhibition Place",0.083333
2,0,"Business reply mail Processing Centre, South C...",0.000000
3,2,"CN Tower, King and Spadina, Railway Lands, Har...",0.062500
4,1,Central Bay Street,0.169231


#### Merge dataframes to include latitude/longitude for each neighbourhood

In [197]:
toronto_coffee = toronto_coffee.join(toronto_venues.set_index("Neighbourhood"), on="Neighbourhood")
print(toronto_coffee.shape)

(1638, 9)


#### Sort results by cluster labels

In [198]:
# sort the results by Cluster Labels
print(toronto_coffee.shape)
toronto_coffee.sort_values(["Cluster Labels"], inplace=True)
toronto_coffee.head()

(1638, 9)


,Cluster Labels,Neighbourhood,Coffee Shop,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
38,0,"University of Toronto, Harbord",0.0,43.662696,-79.400049,Robert Gill Theatre,43.659571,-79.397050,Theater
16,0,"India Bazaar, The Beaches West",0.0,43.668999,-79.315572,The Burger's Priest,43.666731,-79.315556,Fast Food Restaurant
16,0,"India Bazaar, The Beaches West",0.0,43.668999,-79.315572,Brett's Ice Cream,43.667222,-79.312831,Ice Cream Shop
16,0,"India Bazaar, The Beaches West",0.0,43.668999,-79.315572,British Style Fish & Chips,43.668723,-79.317139,Fish & Chips Shop
15,0,"High Park, The Junction South",0.0,43.661608,-79.464763,La Revolucion,43.665460,-79.464308,Mexican Restaurant


## Analysis of Clusters

### Number of neighbourhoods in each cluster

In [199]:
coffee["Cluster Labels"] = kmeans.labels_
coffee.head()

C:\Users\sabee\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Cluster Labels,Neighbourhood,Coffee Shop
0,2,Berczy Park,0.087719
1,2,"Brockton, Parkdale Village, Exhibition Place",0.083333
2,0,"Business reply mail Processing Centre, South C...",0.000000
3,2,"CN Tower, King and Spadina, Railway Lands, Har...",0.062500
4,1,Central Bay Street,0.169231


In [200]:
# How many neighborhoods in each cluster
coffee[['Cluster Labels','Neighbourhood']].groupby('Cluster Labels').count()

,Neighbourhood
Cluster Labels,
0,12
1,10
2,16
3,1


### Average occurences of Coffee Shops in each cluster

In [201]:
# Average Coffee Shops per Cluster
coffee[['Cluster Labels','Coffee Shop']].groupby('Cluster Labels').mean()

,Coffee Shop
Cluster Labels,
0,0.000000
1,0.131837
2,0.073694
3,0.250000


## Mapping Clusters

In [202]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [203]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = ['red', 'purple', 'green','blue'] #Cluster 0 = Blue, Cluster 1 = Red, Cluster 2 = Purple, Cluster 3 = Green
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_coffee['Neighbourhood Latitude'], toronto_coffee['Neighbourhood Longitude'], toronto_coffee['Neighbourhood'], toronto_coffee['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

print('Cluster 0 = Blue, Cluster 1 = Red, Cluster 2 = Purple, Cluster 3 = Green') 
map_clusters

Cluster 0 = Blue, Cluster 1 = Red, Cluster 2 = Purple, Cluster 3 = Green


## Analysis of Individual clusters

#### Creating a dataframe with the borough of each neighbourhood which we will merge with each cluster dataframe

In [204]:
toronto_cluster = toronto_data[['Borough', 'Neighbourhood']]
toronto_cluster.head()

,Borough,Neighbourhood
37,East Toronto,The Beaches
41,East Toronto,"The Danforth West, Riverdale"
42,East Toronto,"India Bazaar, The Beaches West"
43,East Toronto,Studio District
44,Central Toronto,Lawrence Park


### Cluster 1 analysis

In [205]:
cluster1 = toronto_coffee.loc[toronto_coffee['Cluster Labels'] == 0]
toronto_cluster1 = pd.merge(toronto_cluster, cluster1, on='Neighbourhood')
print('There are {} uniques categories.'.format(len(toronto_cluster1['Venue Category'].unique())))
toronto_cluster1[['Borough','Venue Category']].groupby('Venue Category').count().sort_values('Borough', ascending = False).head()

There are 78 uniques categories.


,Borough
Venue Category,
Park,10
Café,8
Bakery,5
Restaurant,4
Sandwich Place,4


In [241]:
toronto_cluster1[['Neighbourhood','Coffee Shop']].groupby('Neighbourhood').mean()

,Coffee Shop
Neighbourhood,
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",0.0
Davisville North,0.0
"Dufferin, Dovercourt Village",0.0
"Forest Hill North & West, Forest Hill Road Park",0.0
"High Park, The Junction South",0.0
"India Bazaar, The Beaches West",0.0
Lawrence Park,0.0
"Moore Park, Summerhill East",0.0
Rosedale,0.0


In [254]:
toronto_cluster1[['Borough','Coffee Shop']].groupby('Borough').mean()

,Coffee Shop
Borough,
Central Toronto,0.0
Downtown Toronto,0.0
East Toronto,0.0
West Toronto,0.0


### Cluster 2 analysis

In [219]:
cluster2 = toronto_coffee.loc[toronto_coffee['Cluster Labels'] == 1]
toronto_cluster2 = pd.merge(toronto_cluster, cluster2, on='Neighbourhood')
print('There are {} uniques categories.'.format(len(toronto_cluster2['Venue Category'].unique())))
toronto_cluster2[['Borough','Venue Category']].groupby('Venue Category').count().sort_values('Borough', ascending = False).head()

There are 157 uniques categories.


,Borough
Venue Category,
Coffee Shop,95
Café,38
Hotel,29
Restaurant,28
Japanese Restaurant,20


In [251]:
toronto_cluster2[['Neighbourhood','Coffee Shop']].groupby('Neighbourhood').mean()

,Coffee Shop
Neighbourhood,
Central Bay Street,0.169231
Church and Wellesley,0.105263
"Commerce Court, Victoria Hotel",0.140000
"First Canadian Place, Underground city",0.130000
"Harbourfront East, Union Station, Toronto Islands",0.120000
"North Toronto West, Lawrence Park",0.105263
"Regent Park, Harbourfront",0.170213
Stn A PO Boxes,0.113402
"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",0.125000


In [255]:
toronto_cluster2[['Borough','Coffee Shop']].groupby('Borough').mean()

,Coffee Shop
Borough,
Central Toronto,0.114286
Downtown Toronto,0.132847


### Cluster 3 analysis

In [223]:
cluster3 = toronto_coffee.loc[toronto_coffee['Cluster Labels'] == 2]
toronto_cluster3 = pd.merge(toronto_cluster, cluster3, on='Neighbourhood')
print('There are {} uniques categories.'.format(len(toronto_cluster3['Venue Category'].unique())))
toronto_cluster3[['Borough','Venue Category']].groupby('Venue Category').count().sort_values('Borough', ascending = False).head()

There are 186 uniques categories.


,Borough
Venue Category,
Coffee Shop,54
Café,44
Restaurant,21
Italian Restaurant,19
Pizza Place,18


In [258]:
toronto_cluster3[['Neighbourhood','Coffee Shop']].groupby('Neighbourhood').mean()

,Coffee Shop
Neighbourhood,
Berczy Park,0.087719
"Brockton, Parkdale Village, Exhibition Place",0.083333
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",0.062500
Christie,0.058824
Davisville,0.058824
"Garden District, Ryerson",0.070000
"Kensington Market, Chinatown, Grange Park",0.060606
"Little Portugal, Trinity",0.066667
"Parkdale, Roncesvalles",0.071429


In [257]:
toronto_cluster3[['Borough','Coffee Shop']].groupby('Borough').mean()

,Coffee Shop
Borough,
Central Toronto,0.074074
Downtown Toronto,0.069672
East Toronto,0.085366
West Toronto,0.075630


### Cluster 4 analysis

In [260]:
cluster4 = toronto_coffee.loc[toronto_coffee['Cluster Labels'] == 3]
toronto_cluster4 = pd.merge(toronto_cluster, cluster4, on='Neighbourhood')
toronto_cluster4[['Borough','Venue Category']].groupby('Venue Category').count().sort_values('Borough', ascending = False).head()

,Borough
Venue Category,
Coffee Shop,9
Arts & Crafts Store,1
Japanese Restaurant,1
Vegetarian / Vegan Restaurant,1
Theater,1


In [244]:
print('There are {} uniques categories.'.format(len(toronto_cluster4['Venue Category'].unique())))
toronto_cluster4[['Neighbourhood','Coffee Shop']].groupby('Neighbourhood').mean()

There are 28 uniques categories.


,Coffee Shop
Neighbourhood,
"Queen's Park, Ontario Provincial Government",0.25


In [259]:
toronto_cluster4[['Borough','Coffee Shop']].groupby('Borough').mean()

,Coffee Shop
Borough,
Downtown Toronto,0.25
